In [64]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.wkt import loads

In [197]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コード : {finish_pref}')





#座標変換
def crs_trf(gdf, dst_proj):
  if gdf.crs is None:
      gdf.set_crs(epsg=dst_proj, inplace=True)
  else:
      gdf.to_crs(epsg=dst_proj, inplace=True)
 
 
      
      

#浸水深スケーリング
def flood_rank_scaling(column):
    flood_rank = np.where((column == 11) | (column == 21), 1, 
                    np.where((column == 12) | (column == 22), 2, 
                        np.where((column == 13) | (column == 23), 3,
                            np.where((column == 14) | (column == 24) | (column == 25) | (column == 26), 4, 5))))
    return flood_rank


     


#最大―最小スケーリング
def min_max_scaling_numpy(column):
    #指定のカラムをnumpy配列に
    array = column.to_numpy()
    
    #最大―最小スケーリングを実施
    min_val = np.min(array)
    max_val = np.max(array)
    scaled_array = (array - min_val) / (max_val - min_val)
    return scaled_array





#重みの算出
def weight(residential_map):
    residential_map['scaled_sum'] = residential_map['estimated_age_scaled'] + residential_map['flood_rank_scaled']
    residential_map['weight'] = residential_map['scaled_sum'] / residential_map['scaled_sum'].sum()




def simulate_subsidy(burden_df, col, frac, num_simulations):
    #試行回数分の出力した値を集計
    simulations = {'house_subsidy': [], 
                   'land_subsidy': [], 
                   'total_subsidy': [], 
                   'house_damage_cost': [], 
                   'assets_damage_cost': [], 
                   'total_damage_cost': [], 
                   'availability': []}

    for _ in range(num_simulations):  # 可変のシミュレーション回数
        relocation_housse = burden_df.relocation_housse(frac=frac, replace=True, weights=burden_df['weight'])
        house_subsidy = (relocation_housse['house_price'] * col).sum()
        land_subsidy = (relocation_housse['land_price'] * col).sum()
        total_subsidy = house_subsidy + land_subsidy
        
        house_damage_cost = relocation_housse['house_damage_price'].sum()
        assets_damage_cost = relocation_housse['assets_damage_price'].sum()
        total_damage_cost = house_damage_cost + assets_damage_cost
        
        #経済的有効性を評価
        availability = total_damage_cost - total_subsidy
        
        for key, value in zip(simulations.keys(), [house_subsidy, land_subsidy, total_subsidy, 
                                                    house_damage_cost, assets_damage_cost, 
                                                    total_damage_cost, availability]):
            simulations[key].append(value)
    
    return {k: np.mean(v) for k, v in simulations.items()}



def subsidy_average(burden_df, num_simulations):
    subsidy_dic = {0.1: 0.05, 
                   0.2: 0.07, 
                   0.3: 0.08, 
                   0.4: 0.1, 
                   0.5: 0.12, 
                   0.6: 0.14, 
                   0.7: 0.17, 
                   0.8: 0.18, 
                   0.9: 0.19, 
                   1.0: 0.21}
    results = []

    for col, frac in subsidy_dic.items():
        #試行を繰り返し出力値を収束させる
        Converged_results = simulate_subsidy(burden_df, col, frac, num_simulations)
        #辞書の値をdfにするためにリストにまとめる．
        results.append([f"{col * 100}%", 
                        f"{frac * 100}%", 
                        Converged_results['house_subsidy'], 
                        Converged_results['land_subsidy'], 
                        Converged_results['total_subsidy'], 
                        Converged_results['house_damage_cost'], 
                        Converged_results['assets_damage_cost'], 
                        Converged_results['total_damage_cost'], 
                        Converged_results['availability']])
    
    return pd.DataFrame(results, columns=['subsidy_ratio', 
                                          'transfer_ratio', 
                                          'house_subsidy', 
                                          'land_subsidy', 
                                          'total_subsidy', 
                                          'house_damage_cost', 
                                          'assets_damage_cost', 
                                          'total_damage_cost', 
                                          'availability'])

    










%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
pref = int(start_pref)
while pref <= int(finish_pref):
    
    
    
    
    
    #市区町村ポリゴンを取得
    total_city_polygon_path = './data/市区町村ポリゴン/japan_ver84/japan_ver84.shp'
    total_city_polygon = gpd.read_file(total_city_polygon_path)
    city_polygon =  total_city_polygon[total_city_polygon["JCODE"].str[:2] == f'{pref:02}']
    
    # NaNを空文字列で置き換える
    city_polygon['SICHO'].fillna('', inplace=True)
    city_polygon['GUN'].fillna('', inplace=True)
    city_polygon['SEIREI'].fillna('', inplace=True)
    city_polygon['SIKUCHOSON'].fillna('', inplace=True)
    #群，政令市と市区町村名を結合
    city_polygon['city'] = city_polygon['SICHO'] + city_polygon['GUN'] + city_polygon['SEIREI']+ city_polygon['SIKUCHOSON']
    
    
    
    
    
    
    #地価価格の読み込み
    land_price_path = f'./data/土地情報センター/{pref}.csv'
    land_price = pd.read_csv(land_price_path, encoding='shift-jis')

    
    
    
    
    
    #建物の損害額
    burden_path = f'./dev/建物家財損害額/{pref}_lost.csv'
    burden_df = pd.read_csv(burden_path, encoding='shift-jis')
    burden_df['geometry'] = loads(burden_df['geometry'])
    burden_df = gpd.GeoDataFrame(burden_df, geometry='geometry', crs=4326)
    
    
    
    
    
    
    
    
    
    
    
    #市区町村ごとに集計
    
    for index, row in city_polygon.iterrows():
        
        #該当の市区町村内の建物データを抽出する
        row_df = pd.DataFrame([row.values], columns=row.index)
        row_df = gpd.GeoDataFrame(row_df, geometry='geometry', crs=4326)
        city_bldg = gpd.sjoin(burden_df, row_df[['JCODE', 'geometry']], op='within')
        city_bldg.rename(columns={'JCODE':'city_code'}, inplace=True)
        city_bldg.drop(columns=['index_right'], axis=1, inplace=True)

        
        
        
        if not city_bldg.empty:
            #該当の市区町村価格を選択
            
            #土地価格から市区町村ポリゴンの市区町村名と等しい土地価格を取得する
            city_name = row_df['city'].iloc[0]
            
            if (land_price['地区名'] == city_name).any():
                #該当の市区町村の土地価格がdfで出力されるので['住宅地']の数値を取得するためにiatを使用
                city_land_price = land_price[land_price['地区名'] == city_name].iat[0, 1]
            
            else:
                #該当の市区町村価格が存在しない場合，都道府県の平均土地価格を使用
                city_land_price = land_price.iat[0, 1]
            
            #土地価格が文字列なのでカンマと型の変更を行う
            city_land_price = city_land_price.replace(',', '')
            city_land_price = int(city_land_price)
            #建物データに土地価格のカラムを追加
            city_bldg['land_price'] = city_bldg['AREA'] * city_land_price
            
            
            
            
            #浸水ランクが5段階と7段階があるので5段かにスケーリングする
            city_bldg['flood_rank'] = flood_rank_scaling(city_bldg['A31_001'])

            #最大―最小スケーリング
            city_bldg['estimated_age_scaled'] = min_max_scaling_numpy(city_bldg['estimated_age'])
            city_bldg['flood_rank_scaled'] = min_max_scaling_numpy(city_bldg['flood_rank'])

            #重みを追加
            weight(city_bldg)

        
        
        
        
            #補助金の算出
            #試行回数を設定
            num_simulations = 100
            subsidy_df = subsidy_average(burden_df, num_simulations)
        
        
        
        
        
        # ディレクトリのパスを構築
        directory_path = f'./dev/補助金/{pref}'
        # ディレクトリが存在しない場合に作成
        os.makedirs(directory_path, exist_ok=True)

        # ファイルパスを構築
        
        #市区町村コードを取得
        city_code = row_df['JCODE'].iloc[0]
        subsidy_df_path = f'{directory_path}/{city_code}.csv'
        # CSVとして保存
        subsidy_df.to_csv(subsidy_df_path, index=False)
    
    
    
    
    
    pref += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コード : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\3886134297.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_polygon['SICHO'].fillna('', inplace=True)
C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\3886134297.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_polygon['GUN'].fillna('', inplace=True)
C:\Users\imai_14\AppData\Local\Temp\ipykernel_5100\3886134297.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [ ]:
def simulate_subsidy(burden_df, col, frac, num_simulations):
    #試行回数分の出力した値を集計
    simulations = {'house_subsidy': [], 
                   'land_subsidy': [], 
                   'total_subsidy': [], 
                   'house_damage_cost': [], 
                   'assets_damage_cost': [], 
                   'total_damage_cost': [], 
                   'availability': []}

    for _ in range(num_simulations):  # 可変のシミュレーション回数
        relocation_housse = burden_df.relocation_housse(frac=frac, replace=True, weights='weight')
        house_subsidy = (relocation_housse['house_price'] * col).sum()
        land_subsidy = (relocation_housse['land_price'] * col).sum()
        total_subsidy = house_subsidy + land_subsidy
        
        house_damage_cost = relocation_housse['house_damage_price'].sum()
        assets_damage_cost = relocation_housse['assets_damage_price'].sum()
        total_damage_cost = house_damage_cost + assets_damage_cost
        availability = total_damage_cost - total_subsidy
        
        for key, value in zip(simulations.keys(), [house_subsidy, land_subsidy, total_subsidy, 
                                                    house_damage_cost, assets_damage_cost, 
                                                    total_damage_cost, availability]):
            simulations[key].append(value)
    
    return {k: np.mean(v) for k, v in simulations.items()}

def subsidy_average(burden_df, num_simulations=100):
    subsidy_dic = {0.1: 0.05, 0.2: 0.07, 0.3: 0.08, 0.4: 0.1, 0.5: 0.12, 0.6: 0.14, 0.7: 0.17, 0.8: 0.18, 0.9: 0.19, 1.0: 0.21}
    results = []

    for col, frac in subsidy_dic.items():
        avg_results = simulate_subsidy(burden_df, col, frac, num_simulations)
        results.append([f"{col * 100}%", f"{frac * 100}%", avg_results['house_subsidy'], 
                        avg_results['land_subsidy'], avg_results['total_subsidy'], 
                        avg_results['house_damage_cost'], avg_results['assets_damage_cost'], 
                        avg_results['total_damage_cost'], avg_results['availability']])
    
    return pd.DataFrame(results, columns=['subsidy_ratio', 'transfer_ratio', 'house_subsidy', 'land_subsidy', 
                                          'total_subsidy', 'house_damage_cost', 'assets_damage_cost', 
                                          'total_damage_cost', 'availability'])


In [190]:
row_df['JCODE'].iloc[0]

'33681'

In [183]:
subsidy_df

,subsidy_ratio,transfer_ratio,house_subsidy,land_subsidy,total_subsidy,house_damage_cost,assets_damage_cost,total_damage_cost,availability
0,subsidy_10.0%,transfer_5.0%,8.479232e+06,211980.8,8.691213e+06,7.280043e+06,5890000.0,1.317004e+07,4.478831e+06
1,subsidy_20.0%,transfer_7.000000000000001%,1.460326e+07,365081.6,1.496835e+07,2.220853e+07,18535000.0,4.074353e+07,2.577519e+07
2,subsidy_30.0%,transfer_8.0%,3.392218e+07,848054.4,3.477023e+07,2.920863e+07,25310000.0,5.451863e+07,1.974840e+07
3,subsidy_40.0%,transfer_10.0%,8.091597e+07,2022899.2,8.293887e+07,9.750303e+06,13170000.0,2.292030e+07,-6.001856e+07
4,subsidy_50.0%,transfer_12.0%,9.414336e+07,2353584.0,9.649694e+07,3.908618e+07,11400000.0,5.048618e+07,-4.601077e+07
5,subsidy_60.0%,transfer_14.000000000000002%,1.542766e+08,3856915.2,1.581335e+08,3.675643e+07,13110000.0,4.986643e+07,-1.082671e+08
6,subsidy_70.0%,transfer_17.0%,1.777386e+08,4443465.6,1.821821e+08,7.098292e+07,30295000.0,1.012779e+08,-8.090417e+07
7,subsidy_80.0%,transfer_18.0%,2.298337e+08,5745843.2,2.355796e+08,9.787432e+07,29315000.0,1.271893e+08,-1.083902e+08
8,subsidy_90.0%,transfer_19.0%,2.856730e+08,7141824.0,2.928148e+08,1.016213e+08,28770000.0,1.303913e+08,-1.624234e+08
9,subsidy_100.0%,transfer_21.0%,3.247219e+08,8118048.0,3.328400e+08,5.256585e+07,38195000.0,9.076085e+07,-2.420791e+08


In [160]:
city_bldg['estimated_age']

Series([], Name: estimated_age, dtype: int64)

In [149]:
crs_trf(city_polygon, 4326)

city_bldg = gpd.sjoin(burden_df, city_polygon, op='contains')

c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [154]:
burden_df.columns

Index(['tatemon_id', 'x', 'y', 'geometry', 'floor', 'AREA', 'atype2',
       'total_area', 'KEY_CODE', 'estimated_structure', 'estimated_age',
       'ave_saving', 'household_age', 'household_sex', 'spouse_age',
       'spouse_sex', 'm_age0', 'm_age5', 'm_age10', 'm_age15', 'f_age0',
       'f_age5', 'f_age10', 'f_age15', 'm_age20', 'm_age25', 'm_age30',
       'm_age35', 'm_age40', 'm_age45', 'm_age50', 'm_age55', 'm_age60',
       'm_age65', 'm_age70', 'm_age75', 'm_age80', 'm_age85', 'm_age90',
       'm_age95', 'm_age100', 'm_age105', 'f_age20', 'f_age25', 'f_age30',
       'f_age35', 'f_age40', 'f_age45', 'f_age50', 'f_age55', 'f_age60',
       'f_age65', 'f_age70', 'f_age75', 'f_age80', 'f_age85', 'f_age90',
       'f_age95', 'f_age100', 'f_age105', 'num_child', 'num_adult', 'meshcode',
       ' T30_P03_SI', ' T30_P06_SI', ' T50_P02_SI', ' T50_P05_SI',
       ' T50_P10_SI', ' T50_P39_SI', 'A31_001', 'cost', 'dpc_rate', 'age',
       'dmg_rate', 'house_price', 'house_damage_price'

In [147]:
burden_df

,tatemon_id,x,y,geometry,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,...,age,dmg_rate,house_price,house_damage_price,assets,children,adult,add_assets,assets_damage_price,total_lost
0,1573,133.797213,34.672247,POINT (133.79721 34.67225),0,166.805,1364,166.805,331012270,1,...,25.5,0.60,4.270208e+07,4.452973e+06,11500000,2,2,4200000,9420000.0,1.387297e+07
1,1688,133.797788,34.673460,POINT (133.79779 34.67346),0,184.710,1364,184.710,331012270,1,...,15.5,0.40,4.728576e+07,9.415541e+06,11500000,2,2,4200000,6280000.0,1.569554e+07
2,1694,133.797064,34.672907,POINT (133.79706 34.67291),0,102.940,1364,102.940,331012270,1,...,5.0,0.40,2.635264e+07,8.833405e+06,11500000,2,4,5800000,6920000.0,1.575340e+07
3,1701,133.797125,34.672804,POINT (133.79712 34.67280),0,68.600,1364,68.600,331012270,1,...,15.5,0.40,1.756160e+07,3.496866e+06,3000000,0,5,4000000,2800000.0,6.296866e+06
4,1703,133.796789,34.672734,POINT (133.79679 34.67273),0,115.070,1364,115.070,331012270,1,...,5.0,0.40,2.945792e+07,9.874295e+06,3000000,0,3,2400000,2160000.0,1.203429e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231777,19890,133.816148,34.860425,POINT (133.81615 34.86043),0,107.045,1364,107.045,33681059005,1,...,15.5,0.40,2.740352e+07,5.456589e+06,5000000,4,2,6800000,4720000.0,1.017659e+07
231778,19898,133.817725,34.860710,POINT (133.81772 34.86071),0,118.770,1364,118.770,33681059006,1,...,5.0,0.40,3.040512e+07,1.019180e+07,11500000,2,2,4200000,6280000.0,1.647180e+07
231779,19908,133.816064,34.861105,POINT (133.81606 34.86110),0,168.965,1364,168.965,33681059006,1,...,8.0,0.40,4.325504e+07,1.281733e+07,11500000,2,2,4200000,6280000.0,1.909733e+07
231780,19909,133.816188,34.860946,POINT (133.81619 34.86095),0,181.605,1364,181.605,33681059006,1,...,15.5,0.40,4.649088e+07,9.257264e+06,11500000,2,2,4200000,6280000.0,1.553726e+07


In [120]:
city_land_price = land_price[land_price['地区名'] == city_name].iat[0, 1]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [113]:

city_name = row_df['city'].iloc[0]
city_land_price = land_price[land_price['地区名'] == city_name]

In [114]:
city_land_price

,地区名,住宅地,宅地見込地,商業地,工業地,全用途
2,岡山市北区,"82,500",NaN,"265,700","47,800","167,700"


In [90]:
row_df['city']

0    岡山市北区
Name: city, dtype: object

In [124]:
land_price.iat[0, 1]

'45,000'